# Objective 
- 교통사고의 원인을 규명하고 사고율을 낮추기 위해, 

시공간 정보로부터 사고위험도(ECLO)를 예측

- ※ ECLO(Equivalent Casualty Loss Only) : 인명피해 심각도

ECLO = 사망자수 * 10 + 중상자수 * 5 + 경상자수 * 3 + 부상자수 * 1
본 대회에서는 사고의 위험도를 인명피해 심각도로 측정

## Import

In [26]:
import numpy as np
import pandas as pd
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
#ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import linear_kernel
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMClassifier
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import tensorflow as tf
from transformers import pipeline, set_seed, BertTokenizer, TFBertForSequenceClassification
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
import re
import nltk 
from nltk.corpus import stopwords
# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
from sentence_transformers import SentenceTransformer


#### hiddencell

## Fixed Random Seed
- Seed값에 의해 동일한 코드를 사용해도 결과가 다를 수 있기에, 동일한 결과를 위해 seed 값을 고정시킴

In [27]:
import os
import random
import numpy as np

def seed_everything(seed):
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    seed_everything(42)

## Load Data
- train.csv, test.csv 로드해 상위행 출력

In [28]:
# 학습용 데이터 로드

train_path = 'D:\Code\대구 교통사고 피해 예측 AI 경진대회/train.csv '

df_train = pd.read_csv(train_path)

print(f'학습용 데이터 : \n{df_train}')

학습용 데이터 : 
                   ID           사고일시   요일 기상상태              시군구  \
0      ACCIDENT_00000  2019-01-01 00  화요일   맑음     대구광역시 중구 대신동   
1      ACCIDENT_00001  2019-01-01 00  화요일   흐림    대구광역시 달서구 감삼동   
2      ACCIDENT_00002  2019-01-01 01  화요일   맑음    대구광역시 수성구 두산동   
3      ACCIDENT_00003  2019-01-01 02  화요일   맑음     대구광역시 북구 복현동   
4      ACCIDENT_00004  2019-01-01 04  화요일   맑음     대구광역시 동구 신암동   
...               ...            ...  ...  ...              ...   
39604  ACCIDENT_39604  2021-12-31 19  금요일   맑음  대구광역시 수성구 수성동3가   
39605  ACCIDENT_39605  2021-12-31 19  금요일   맑음    대구광역시 달서구 상인동   
39606  ACCIDENT_39606  2021-12-31 21  금요일   맑음    대구광역시 달서구 월성동   
39607  ACCIDENT_39607  2021-12-31 22  금요일   맑음     대구광역시 달서구 장동   
39608  ACCIDENT_39608  2021-12-31 23  금요일   맑음     대구광역시 서구 비산동   

                  도로형태 노면상태  사고유형 사고유형 - 세부분류       법규위반  ... 가해운전자 상해정도  \
0             단일로 - 기타   건조  차대사람  길가장자리구역통행중    안전운전불이행  ...       상해없음   
1             단일로 - 기타   건조  차대사

In [29]:
# 학습용 데이터의 정보 확인하기
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           39609 non-null  object
 1   사고일시         39609 non-null  object
 2   요일           39609 non-null  object
 3   기상상태         39609 non-null  object
 4   시군구          39609 non-null  object
 5   도로형태         39609 non-null  object
 6   노면상태         39609 non-null  object
 7   사고유형         39609 non-null  object
 8   사고유형 - 세부분류  39609 non-null  object
 9   법규위반         39609 non-null  object
 10  가해운전자 차종     39609 non-null  object
 11  가해운전자 성별     39609 non-null  object
 12  가해운전자 연령     39609 non-null  object
 13  가해운전자 상해정도   39609 non-null  object
 14  피해운전자 차종     38618 non-null  object
 15  피해운전자 성별     38618 non-null  object
 16  피해운전자 연령     38618 non-null  object
 17  피해운전자 상해정도   38618 non-null  object
 18  사망자수         39609 non-null  int64 
 19  중상자수         39609 non-nu

In [30]:
train_columns = df_train.columns
print(train_columns)

Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형',
       '사고유형 - 세부분류', '법규위반', '가해운전자 차종', '가해운전자 성별', '가해운전자 연령', '가해운전자 상해정도',
       '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도', '사망자수', '중상자수',
       '경상자수', '부상자수', 'ECLO'],
      dtype='object')


In [31]:
# 각 한글 컬럼들을 영문컬럼으로 변환
df_train.columns=['ID', # ID,
                  'AHD',#사고일시
                  'Day',#요일
                  'Weather',#기상상태
                  'Location',#시군구
                  'LoadShape',#도로형태
                  'LordStatus',#노면상태
                  'Accident',#사고유형
                  'Accident-Detail',#사고유형-세부분류
                  'Violation of law',#법규위반
                  'Driver_kind',#가해운전자 차종
                  'Driver_sex',#가해운전자 성별
                  'Driver_age',#가해운전자 연령
                  'Driver_damaged',#가해운전자 상해정도
                  'Victim_kind',#피해운전자 차종
                  'Victim_sex',#피해운전자 성별
                  'Victim_age',#피해운전자 연령
                  'Victim_damaged',#피해운전자 상해정도
                  'n_Dead',#사망자수
                  'S_injured',#중상자수
                  'm_injured',#경상자수
                  'injured',#부상자수
                  'ECLO' #정답컬럼
]

print(df_train.columns)

Index(['ID', 'AHD', 'Day', 'Weather', 'Location', 'LoadShape', 'LordStatus',
       'Accident', 'Accident-Detail', 'Violation of law', 'Driver_kind',
       'Driver_sex', 'Driver_age', 'Driver_damaged', 'Victim_kind',
       'Victim_sex', 'Victim_age', 'Victim_damaged', 'n_Dead', 'S_injured',
       'm_injured', 'injured', 'ECLO'],
      dtype='object')


In [32]:
# 학습용 데이터의 결측치 확인 및 제거
df_train.isnull().sum()

# 피해운전자 차종,성별,연령,상해정도 총 4개의 컬럼에 결측치가 존재.
# 

ID                    0
AHD                   0
Day                   0
Weather               0
Location              0
LoadShape             0
LordStatus            0
Accident              0
Accident-Detail       0
Violation of law      0
Driver_kind           0
Driver_sex            0
Driver_age            0
Driver_damaged        0
Victim_kind         991
Victim_sex          991
Victim_age          991
Victim_damaged      991
n_Dead                0
S_injured             0
m_injured             0
injured               0
ECLO                  0
dtype: int64

In [33]:
# 결측치가 너무 많아 채울 수가 없는 컬럼 제거

df_train.drop(columns=['Victim_kind',
                       'Victim_sex',
                       'Victim_age',
                       'Victim_damaged'], inplace=True)

print(df_train)

                   ID            AHD  Day Weather         Location  \
0      ACCIDENT_00000  2019-01-01 00  화요일      맑음     대구광역시 중구 대신동   
1      ACCIDENT_00001  2019-01-01 00  화요일      흐림    대구광역시 달서구 감삼동   
2      ACCIDENT_00002  2019-01-01 01  화요일      맑음    대구광역시 수성구 두산동   
3      ACCIDENT_00003  2019-01-01 02  화요일      맑음     대구광역시 북구 복현동   
4      ACCIDENT_00004  2019-01-01 04  화요일      맑음     대구광역시 동구 신암동   
...               ...            ...  ...     ...              ...   
39604  ACCIDENT_39604  2021-12-31 19  금요일      맑음  대구광역시 수성구 수성동3가   
39605  ACCIDENT_39605  2021-12-31 19  금요일      맑음    대구광역시 달서구 상인동   
39606  ACCIDENT_39606  2021-12-31 21  금요일      맑음    대구광역시 달서구 월성동   
39607  ACCIDENT_39607  2021-12-31 22  금요일      맑음     대구광역시 달서구 장동   
39608  ACCIDENT_39608  2021-12-31 23  금요일      맑음     대구광역시 서구 비산동   

             LoadShape LordStatus Accident Accident-Detail Violation of law  \
0             단일로 - 기타         건조     차대사람      길가장자리구역통행중          안전운전불이행   
1

In [34]:
df_train.isnull().sum()

ID                  0
AHD                 0
Day                 0
Weather             0
Location            0
LoadShape           0
LordStatus          0
Accident            0
Accident-Detail     0
Violation of law    0
Driver_kind         0
Driver_sex          0
Driver_age          0
Driver_damaged      0
n_Dead              0
S_injured           0
m_injured           0
injured             0
ECLO                0
dtype: int64

In [35]:
print(df_train)

'''

ECLO = 사망자수 * 10 + 중상자수 * 5 + 경상자수 * 3 + 부상자수 * 1

'''

                   ID            AHD  Day Weather         Location  \
0      ACCIDENT_00000  2019-01-01 00  화요일      맑음     대구광역시 중구 대신동   
1      ACCIDENT_00001  2019-01-01 00  화요일      흐림    대구광역시 달서구 감삼동   
2      ACCIDENT_00002  2019-01-01 01  화요일      맑음    대구광역시 수성구 두산동   
3      ACCIDENT_00003  2019-01-01 02  화요일      맑음     대구광역시 북구 복현동   
4      ACCIDENT_00004  2019-01-01 04  화요일      맑음     대구광역시 동구 신암동   
...               ...            ...  ...     ...              ...   
39604  ACCIDENT_39604  2021-12-31 19  금요일      맑음  대구광역시 수성구 수성동3가   
39605  ACCIDENT_39605  2021-12-31 19  금요일      맑음    대구광역시 달서구 상인동   
39606  ACCIDENT_39606  2021-12-31 21  금요일      맑음    대구광역시 달서구 월성동   
39607  ACCIDENT_39607  2021-12-31 22  금요일      맑음     대구광역시 달서구 장동   
39608  ACCIDENT_39608  2021-12-31 23  금요일      맑음     대구광역시 서구 비산동   

             LoadShape LordStatus Accident Accident-Detail Violation of law  \
0             단일로 - 기타         건조     차대사람      길가장자리구역통행중          안전운전불이행   
1

'\n\nECLO = 사망자수 * 10 + 중상자수 * 5 + 경상자수 * 3 + 부상자수 * 1\n\n'

TypeError: no numeric data to plot

In [ ]:
# 평가용 데이터 생성

test_path = 'D:\Code\대구 교통사고 피해 예측 AI 경진대회/test.csv'

df_test = pd.read_csv(test_path)

print(f'평가용 데이터 : \n{df_test}')

평가용 데이터 : 
                   ID           사고일시   요일 기상상태              시군구         도로형태  \
0      ACCIDENT_39609  2022-01-01 01  토요일   맑음     대구광역시 수성구 상동   교차로 - 교차로안   
1      ACCIDENT_39610  2022-01-01 01  토요일   맑음    대구광역시 수성구 지산동     단일로 - 기타   
2      ACCIDENT_39611  2022-01-01 04  토요일   맑음  대구광역시 수성구 수성동2가   교차로 - 교차로안   
3      ACCIDENT_39612  2022-01-01 04  토요일   맑음    대구광역시 수성구 신매동     단일로 - 기타   
4      ACCIDENT_39613  2022-01-01 06  토요일   맑음    대구광역시 달서구 감삼동   교차로 - 교차로안   
...               ...            ...  ...  ...              ...          ...   
10958  ACCIDENT_50567  2022-12-31 18  토요일   맑음     대구광역시 남구 대명동     단일로 - 터널   
10959  ACCIDENT_50568  2022-12-31 18  토요일   맑음    대구광역시 수성구 시지동     단일로 - 기타   
10960  ACCIDENT_50569  2022-12-31 20  토요일   맑음    대구광역시 수성구 연호동     단일로 - 기타   
10961  ACCIDENT_50570  2022-12-31 20  토요일   맑음    대구광역시 수성구 범물동  교차로 - 교차로부근   
10962  ACCIDENT_50571  2022-12-31 21  토요일   맑음     대구광역시 동구 효목동  교차로 - 교차로부근   

      노면상태  사고유형  
0       건